## 2-Updated Sentiment Analysis

이전 노트북에서 감성분석의 기본적인 것들을 살펴보았고 여기서는 
다음의 것들에 대한 결과를 살펴본다

packed padded sequences 
pre-trained word embeddings 
different RNN architecture
bidirectional RNN
multi-layer RNN
regularization
a different optimizer

## Preparing Data

이전까지와 같이 시드를 배정하고 Fields를 정의하고 train/valid/test 셋을 분리할 것이다. 
RNN이 오직 우리 시퀀스에서 패딩되지 않은 원소만 계산하게 하기 위해서 packed padded sequence를 사용할것이고 패딩된 부분에 대해서는 제로 텐서로 만들 것이다. packed padded sequence를 사용하기 위해서는 RNN에 실제문장길이를 입력해야 한다.

In [4]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True) #include_lengths가 추가된 부분
LABEL = data.LabelField(dtype = torch.float)

IMDb dataset 불러오기

In [5]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

training set으로부터 validation set 만들기

In [10]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [12]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:59, 4.79MB/s]                               
100%|█████████▉| 399854/400000 [00:30<00:00, 22703.64it/s]

In [13]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True, #배치안의 모든 텐서들이 길이를 기준으로 분리됨
    device = device)

# Build the Model

## Different RNN Architectur

Long Short-Term Memory(LSTM)이라고 불리는 RNN 아키텍처를 사용할 것이다.
왜 LSTM이 RNN보다 낫냐. vanishing gradient problem때문이다. LSTM은 cell  $c$ 라고 불리는 recurrent state를 추가함으로써 LSTM의 메모리 역할을 하여 메모리중에서 정보의 흐름을 통제하는 다중 게이트의 기능을 한다. 
식으로 표현하자면 $$(h_t, c_t) = \text{LSTM}(x_t, h_t, c_t)$$ 이다

초기 셀 상태인 $c_0$는 초기 hidden state와 같이 모든 텐서가 0이다. 그러나 여전히 감성 예측은 오직 마지막 셀 상태 $\hat{y}=f(h_T)$.가 아니라 hidden state를 가지고만 이뤄진다.


## Bidirectional RNN

양방향 RNN의 컨셉은 매우 간단하다. RNN이 문장의 단어들이 처음부터 끝까지 정방향의 과정을 거쳤다면 반대로 끝에서부터 시작하는 역방향 RNN이 존재하는 것이다. 정방향에서 $x_t$, 이면 역방향 RNN은 $x_{T-t+1}$.을 계산하고 있다.

파이토치에서 정,역방향에서 계산된 텐서들은 각각 맨 위에 서로다른 단일 텐서로 쌓인다.
우리는 감성 예측을 정방향, 그리고 역방향의 각각의 마지막 hidden state의 연쇄로 구하게 된다.

## Multi-layer RNN

deep RNN이라고도 불리는 Multi-layer RNN 또한 간단한 컨셉으로 이뤄진다. 기존의 일반 RNN에 추가하는것..

## Regularization

비록 모델의 향상을 위한거지만 각각의 방법들은 모델에 매개변수들을 추가시켰다.
매개변수의 수가 많아질수록 모델이 과적합(오버피팅)될 가능성이 커진다. 이를 해결하기 위해서 정규화를 사용한다. 더 특징적으로는 드롭아웃이라는 정규화 방법을 사용한다. 

## Implementation Details

우리는 모델에서 패드 토큰이 임베딩을 하면서 문장의 감성을 판단하는 데 있어서 중요하지 않은 것을 알기 때문에 패드토큰이 padding_idx로 임베딩 레이어를 통과하게 한다.

RNN대신에 LSTM을 사용하기위해서 nn.LSTM을 쓴다. 또한 RNN이 output과 hidden state 를 출력했다면 LSTM은 output과 튜플형태의 hidden state 그리고 final cell state를 반환하는 것에 대해 주목해라.

LSTM의 최종 hidden state가 정방향,역방향 요소를 모두 가지고있기때문에 나중에 연쇄될 것이지만, nn.Linear 층의 입력 사이즈는 은닉층의 두배이다.

드롭아웃은 nn.Dropout을 시행하므로써 실행되며 드롭아웃 시키고 싶은 각각의 레이어들에 대한 정방향 방법에서 사용된다. 입력층과 출력층에서 드룹아웃을 절대 사용하면안된다.( 이 경우에는 text와 fc이다) 중간 층에서만 사용해야하고 LSTM에서는 한층의 은닉상태에서 다음 층의 은닉 상태로 가는데만 사용된다.



In [14]:

import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden.squeeze(0))

이전에 했던 것 과 같이, bidirectionality와 드롭아웃 가능성, 레이어의 수에 대한 논의와 매개변수를 적용하여 RNN클래스의 인스턴스를 만들 것이다.

사전 학습된 벡터들이 모델에 로드되게 하기 위해서 EMBEDDING_DIM은 이전에 로드된 사전 학습된 Glove 벡터들과 동일하게 해야 한다.

단어들에서 패드 토큰의 인덱스를 가져오며 실제 스트링이 <pad> 로 표기된 패트토큰의 특성을
대표하게 해야 한다.

In [17]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT,
            PAD_IDX)

이제 모델의 파라미터 수를 보여줄 것이다.
기존 파라미터들의 수보다 두배 증가했음에 유의해라

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


마지막으로 추가할 것은 사전에 학습된 단어 임베딩을 복사하여 모델의 임베딩 레이어에 넣는 것이다. filed의 vocab에서 임베딜을 검색하여 사이즈가 정확한지에 대하여 검색한다.

In [21]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [22]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.1181,  0.5395,  0.1928,  ...,  0.3824, -0.7506, -0.3359],
        [-0.6320,  0.2957,  0.2505,  ..., -0.0964, -0.2494,  0.3276],
        [-0.1673, -0.5686,  0.2724,  ..., -0.9791,  0.1121,  0.3235]])

<unk> 그리고 <pad> 토큰들이 사전 학습된 단어들 속에 있지 않기 때문에 vocab 을 만들 떄 unk_init을 사용한다. 이들 모두를 0으로 만드는 것이 선호되는데 어차피 감성에 대한 결정을 하는데 있어 의미있게 작용하지 않기 때문이다.

이것은 임베딩 가중치 행렬의 열을 0으로 수동적으로 바꾸면서 진행된다.

In [23]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.1181,  0.5395,  0.1928,  ...,  0.3824, -0.7506, -0.3359],
        [-0.6320,  0.2957,  0.2505,  ..., -0.0964, -0.2494,  0.3276],
        [-0.1673, -0.5686,  0.2724,  ..., -0.9791,  0.1121,  0.3235]])


# Train the Model

변화된 것 한가지는 옵티마이저가 SGD 에서 Adam으로 바뀌는 것이다. SGD는 모든 파라미터들을 같은 learning rate로 업데이트 시킨다.

SGD를 Adam 으로 바꾸기 위해서 우리는 간단히 optim.SGD를 optim.Adam으로 바꾸면 된다. 또한 아담은 초기 러닝레이트를 제공할 필요 없이 파이토치가 값을 매겨준다.

In [24]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

나머지 모델 훈련과정은 다르지 않다.
우리는 criterion을 정의하고 모델과 criterion을 gpu에 싣는다.

In [25]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [26]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [27]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

그리고 모델을 테스트 하기위해서 함수를 정의하는데 batch.text를 분리하는걸 기억한다.
Note : 드롭아웃을 사용하기 때문에, model.eval()을 사용해 evaluating 하는동안 turn off 해야한다.

In [28]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

그리고 에폭이 도는데 얼마나 걸리는지 알려주는 함수를 생성한다

In [29]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

모델을 학습시킨다

In [30]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 55s
	Train Loss: 0.643 | Train Acc: 62.02%
	 Val. Loss: 0.495 |  Val. Acc: 77.49%
Epoch: 02 | Epoch Time: 0m 55s
	Train Loss: 0.534 | Train Acc: 73.11%
	 Val. Loss: 0.467 |  Val. Acc: 77.91%
Epoch: 03 | Epoch Time: 0m 55s
	Train Loss: 0.433 | Train Acc: 80.34%
	 Val. Loss: 0.365 |  Val. Acc: 84.24%
Epoch: 04 | Epoch Time: 0m 55s
	Train Loss: 0.370 | Train Acc: 84.08%
	 Val. Loss: 0.450 |  Val. Acc: 81.69%
Epoch: 05 | Epoch Time: 0m 54s
	Train Loss: 0.316 | Train Acc: 87.11%
	 Val. Loss: 0.293 |  Val. Acc: 87.80%


이제 향상된 테스트 셋의 정확도를 살펴보자!

In [31]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | 
      Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.307 | Test Acc: 87.29%


우리는 부정적인 리뷰의 값이 0에 가깝고 긍정적인 리뷰가 1에 가까운 값이 나오도록 하고싶다.

In [32]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [43]:
predict_sentiment(model, "This film is terrible")

0.0074930922128260136

In [40]:
predict_sentiment(model, "This film is great")

0.9873881936073303

## Next Steps

우리는 이제껏 영화 리뷰에 대한 감성 분석기를 만들었다! 다음 노트에서는 더 적은 파라미터들을 가지고 학습하여 비교적 정확성있고 빠른 모델을 만들도록 해보자